In [88]:
import os
import csv
import pymysql as mc 
import time
import webbrowser
import sys
from sklearn.cluster import KMeans
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.preprocessing import scale
import matplotlib as mpl
mpl.rc('figure',figsize=[12,8])
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR
from sklearn.tree import DecisionTreeClassifier
import getpass
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Imputer 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [89]:
#credentials
user='akashgoyal'
host='stromberg.cs.uchicago.edu'
db_name = 'mlb_practicum'
pw = getpass.getpass()

········


In [158]:
cnx2 = create_engine('mysql+pymysql://'+user+':'+pw+'@'+host+':3306/'+db_name,echo=False)
cnx = mc.connect(user=user,password=pw,
                 host=host,db=db_name,port=3306)

In [179]:
start = time.time()
rel_cols = ['Pitch2.pitcherID','Pitch2.batterHand','Pitch2.pitcherHand','the_event','akash','curr_inn','balls',
            'strikes','cumulativePitches','timesFaced','home_team_runs','away_team_runs',
            'outs','firstBaseRunner','secondBaseRunner',
            
            'swing_four_seam_rate',
            'whiff_four_seam_rate','ball_four_seam_rate','whiff_slider_rate',
            'swing_slider_rate','ball_slider_rate','whiff_two_seam_rate',
            'swing_two_seam_rate','ball_two_seam_rate','whiff_change_up_rate',
            'swing_change_up_rate','ball_change_up_rate','whiff_knuckleball_rate',
            'swing_knuckleball_rate','ball_knuckleball_rate','whiff_curve_rate',
            'swing_curve_rate','ball_curve_rate','whiff_cutter_rate','ball_cutter_rate',
            'swing_cutter_rate', 
            
            #distribution of pitches seen by the team and by the individual in current game
             'psfg.pct_four_seam_team','psfg.pct_curve_team','psfg.pct_slider_team',
             'psfg.pct_two_seam_team','psfg.pct_cutter_team',
             'psfg.pct_knuckleball_team','psfg.pct_sinker_team','psfg.pct_change_up_team','psfg.pct_four_seam_ind',
             'psfg.pct_curve_ind','psfg.pct_slider_ind','psfg.pct_two_seam_ind','psfg.pct_cutter_ind',
             'psfg.pct_knuckleball_ind','psfg.pct_sinker_ind','psfg.pct_change_up_ind',
            
             #does leverage play a role?
             'win_probability','leverage',
            
             #batter run expectancy
             '100_avg','300_avg','500_avg','1000_avg','2000_avg',
              
              
              #in-inning distribution of pitches seen (the original mistake I made that is ok)
              'inn.pct_four_seam_ind AS AB_four_seam','inn.pct_curve_ind AS AB_curve_team',
              'inn.pct_slider_ind AS AB_slider','inn.pct_two_seam_ind AS AB_two_seam',
              'inn.pct_cutter_ind AS AB_cutter','inn.pct_knuckleball_ind AS AB_knuckleball',
              'inn.pct_sinker_ind AS AB_sinker','inn.pct_change_up_ind AS AB_change_up',
                
                
              #from player table
             'batPosition','hr','rbi']

             #this is prior distribution of pitches seen by the batter- may need to comment this out
             #'bc.change_up','bc.curve','bc.cutter','bc.four_seam','bc.knuckleball',
            #'bc.sinker','bc.slider','bc.two_seam',
            
            #'four_seam_slug','slider_slug','change_up_slug','two_seam_slug','sinker_slug','knuckleball_slug',
           #'curve_slug','cutter_slug']

query = '''SELECT {} FROM Pitch2 INNER JOIN pitcher_ms_clusters_5_26_18 pms1
            ON pms1.playerID = Pitch2.pitcherID 
            INNER JOIN Distinct_Players p1 ON p1.playerID = pms1.playerID
            INNER JOIN Pitch_type_mapping pt1 ON pt1.original = Pitch2.pitchType
            INNER JOIN Game ON Game.gameID = Pitch2.gameID
            INNER JOIN batter_whiffs bw ON (bw.batterID = Pitch2.batterID
            AND bw.gameDate = Game.gameDate AND bw.pitcherHand = Pitch2.pitcherHand)
            INNER JOIN Pitches_Seen_Full_Game psfg ON 
            (psfg.gameID = Game.gameID AND psfg.pitchID = Pitch2.pitchID)
            INNER JOIN Batter_Run_Expectancy bre ON 
            (bre.playerID = Pitch2.batterID AND bre.gameID = Game.gameID)
            INNER JOIN Player ON (Player.playerID = Pitch2.batterID AND Pitch2.gameID = Player.gameID)
            INNER JOIN leverage l1 ON (l1.gameID = Pitch2.gameID AND l1.pitchID = Pitch2.pitchID)
            INNER JOIN Pitches_Seen inn ON (inn.gameID = Pitch2.gameID AND inn.pitchID = Pitch2.pitchID)
            
            WHERE Pitch2.pitcherID = 285079 AND
            YEAR(Game.gameDate)!=2016'''.format(','.join(rel_cols))
df = pd.read_sql_query(query,cnx)
print (f'that took {time.time()-start} seconds')
#pms1.cls_bandwidth_8=5 ADD THIS BACK INTO THE WHERE CLASS LASTER
#INNER JOIN pitcher_classifications_5_25_18 pc5 ON (pc5.playerID = Pitch2.pitcherID AND 
#                pc5.gameDate = Game.gameDate)
#'pc5.change_up','pc5.curve','pc5.cutter','pc5.four_seam','pc5.knuckleball',
#               'pc5.sinker','pc5.slider','pc5.two_seam',

that took 13.483497858047485 seconds


In [180]:
c_df = df.copy()
df.shape

(18578, 70)

In [181]:
#gets the unique values and their counts
df['pitcherID'].value_counts()

285079    18578
Name: pitcherID, dtype: int64

In [137]:
#choose just zach greinke
c_df = df[df['pitcherID']=='461872']
#zach greinke throws 13.4% change ups, 11.9% curveballs, 3.1% cutters, 19.3% sliders
#19.5% two-seamers

In [182]:
#preprocessing data to get certain statistics
start = time.time()
def platoon_advantage(df):
    if df.loc['batterHand'] == df.loc['pitcherHand']:
        return 1
    else:
        return 0

def get_current_lead(df):
    homeLead = df.loc['home_team_runs']-df.loc['away_team_runs']
    if df.loc['curr_inn']*2%2==1:
        return homeLead
    else:
        return -homeLead
    
def advantageous_runner(df):
    if not df.loc['firstBaseRunner'] or df.loc['secondBaseRunner']:
        return 0
    else:
        return 1
c_df['batPosition'] = c_df['batPosition'].fillna(value=9) #override pinch hitters w/ 7th spot
c_df['advantage_runner'] = c_df.apply(advantageous_runner,axis=1)
c_df['platoon_advantage'] = c_df.apply(platoon_advantage,axis=1)
c_df['current_lead_deficit'] = c_df.apply(get_current_lead,axis=1)
c_df['count'] = c_df['balls'].map(str)+c_df['strikes'].map(str)
c_df = pd.get_dummies(c_df,columns=['count']) #create 12 dummy variables for count type

#now that we're done applying stuff eliminate the bad columns
dropped_columns = ['pitcherID','batterHand','pitcherHand','home_team_runs','away_team_runs',
                  'firstBaseRunner','secondBaseRunner','the_event','balls','strikes']
c_df = c_df.drop(columns=dropped_columns)
print (f'that took {time.time()-start} seconds')

that took 7.82328987121582 seconds


In [183]:
shuffled_data = shuffle(c_df)
first_half = int(shuffled_data.shape[0]/2)
train_data = shuffled_data.iloc[:first_half]
test_data = shuffled_data.iloc[first_half:]
X = train_data.drop(columns=['akash'])
Y = train_data['akash']
X_oos = test_data.drop(columns=['akash'])
Y_oos = test_data['akash']
x_headers = X.columns.values

In [184]:
#fill in NaN values with the mean for each column. Probably not the greatest method
#because of survivorship but not many options at this point
imp = Imputer(strategy='mean') 
X = imp.fit_transform(X)
X = pd.DataFrame(X,columns=x_headers) #imputer transforms it to numpy
imp2 = Imputer(strategy='mean')
X_oos = imp2.fit_transform(X_oos)
X_oos = pd.DataFrame(X_oos,columns=x_headers) 

In [185]:
dt = DecisionTreeClassifier(criterion='entropy',min_impurity_decrease=0.01)
dt.fit(X,Y)
dt.score(X,Y)

0.8512218753364195

In [186]:
dt.score(X_oos,Y_oos)

0.853913230702982

In [187]:
Y_oos.value_counts(normalize=True)

knuckleball    0.846808
four_seam      0.153192
Name: akash, dtype: float64

In [188]:
Y_oos.value_counts(normalize=True).iloc[0]

0.8468080525352567

In [171]:
actual_v_expected = pd.DataFrame(np.column_stack((Y_oos,dt.predict(X_oos))))
(actual_v_expected[actual_v_expected[1]!='knuckleball'])

,0,1
30,knuckleball,four_seam
59,four_seam,four_seam
153,knuckleball,four_seam
155,four_seam,four_seam
191,knuckleball,four_seam
205,four_seam,four_seam
230,four_seam,four_seam
303,four_seam,four_seam
305,knuckleball,four_seam
427,four_seam,four_seam


In [144]:
ctr = 0
tot = 0
for row in actual_v_expected.itertuples():
    if row[2]=='slider':
        if row[1]==row[2]:
            ctr+=1
        tot+=1
ctr, tot,ctr/tot

ZeroDivisionError: division by zero

In [172]:
avg_nodes = int((X.shape[1]+8)/2)
clf = MLPClassifier(solver='adam',alpha=0.001,hidden_layer_sizes=(avg_nodes,),random_state=1)

In [173]:
clf.fit(X,Y)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(41,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [174]:
clf.score(X,Y)

0.86543223167187

In [175]:
clf.score(X_oos,Y_oos)

0.8603724835827322

In [150]:
probabilities = clf.predict_proba(X_oos)
probabilities = np.amax(probabilities,axis=1)
guesses = clf.predict(X_oos)
comb = np.hstack((probabilities.reshape(-1,1),guesses.reshape(-1,1),Y_oos.reshape(-1,1)))
comb_df = pd.DataFrame(comb,columns=['max probability','guess','actual output'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [151]:
comb_df[comb_df['max probability']<0.3]

,max probability,guess,actual output
787,0.298541,four_seam,slider
2877,0.285039,four_seam,four_seam


In [176]:
logit.fit(X,Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [177]:
logit.score(X,Y)

0.8615566799440199

In [178]:
logit.score(X_oos,Y_oos)

0.8567122402842071